# Rewrite `_base` function to fit more jobs.
Also, start learning to simplify what is needed.

In [1]:
# Import relevant modules
import os           # interface with the underlying OS
import numpy as np  # linear algebra
import pandas as pd # dataframe
import re           # regular expression

In [2]:
# Define path to look at
path = 'CityofLA/Job Bulletins/'

# Get a list of all txt files in this path
all_txt_files = os.listdir(path)
all_txt_files.sort() # WARNING: this mutates the list

In [3]:
# Do some checks here
print(len(all_txt_files))      # length should be 683 as manually verified
print(len(set(all_txt_files))) # see if each file is unique. hopefully 683 as well!
print('SYSTEMS ANALYST 1596 102717.txt' in all_txt_files) # should be True

683
683
True


In [4]:
# Find `SYSTEMS ANALYST 1596 102717.txt` (which exists).
print(all_txt_files.index('SYSTEMS ANALYST 1596 102717.txt'))

609


## Import file as text
`sa` stands for systems analyst

In [5]:
sa_path = path + all_txt_files[609]
sa = open(sa_path, 'rt').read()
sa # display

'SYSTEMS ANALYST\n\nClass Code: 1596\nOpen Date: 10-27-17\n(Exam Open to All, including Current City Employees)\n\nANNUAL SALARY\n\n$68,611 to $100,307\nThe salary in the Department of Water and Power is $70,908 to $88,092 and $83,770 to $104,065.\n\nNOTES:\n\n1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.  \n2. Annual salary is at the start of the pay range. The current salary range is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer.\n3. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.\n\n\nDUTIES\n\nA Systems Analyst analyzes procedures, methods and operations of computer-based information systems; designs, implements, and recommends information systems to improve the efficiency and economy of City operations; performs cost benefit and feasibility analyses re

## First, do individual parts 

<font color='red'> Need to use regex for a better information location. </font>

In [6]:
# Locate the information
temp = sa[sa.find('REQUIREMENTS/MINIMUM QUALIFICATIONS'):sa.find('PROCESS NOTES')]
temp

'REQUIREMENTS/MINIMUM QUALIFICATIONS\n\n1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or\n2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:\na. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or\nb. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or\nc. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or\n3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and\na. Satisfactory completion of four cou

In [7]:
# Get the digits
digits_as_str = [sentence[-1] for sentence in temp.split('.') if sentence[-1].isdigit()]
digits_as_str

['1', '2', '3']

In [8]:
# Separate requirements one from the other.
## Initialization
line_item = [] # borrow from accounting term. means items in a list
## Loop through idx of digits_as_str to get the text from one digit to the other
for idx in range(len(digits_as_str)-1):
    start = digits_as_str[idx]  ; pos1 = temp.index(start)
    end   = digits_as_str[idx+1]; pos2 = temp.index(end)
    item  = temp[pos1:pos2]     ; line_item.append(item)
## Put the last part into the list of all possible requirements
final_dig    = digits_as_str[-1]; final_pos = temp.index(final_dig)
final_item = temp[final_pos:] ; line_item.append(final_item)
## Display results
line_item

['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or\n',
 '2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:\na. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or\nb. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or\nc. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or\n',
 '3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and\na. Satisfactory completion of four courses, of at least three seme

In [9]:
# Split further by newline characters
line_item = [[line for line in item.split('\n') if len(line) > 1] for item in line_item]
line_item

[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or'],
 ['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
  'a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or',
  'b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or',
  'c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or'],
 ['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and',
  'a. Satisfactory completion of four courses, of at

In [10]:
# # Remove one layer of nested list.
# # Source: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
# line_item = [[item for sublist in l for item in sublist] for l in line_item]
# line_item

In [11]:
# Simplify line_item by throwing away anything after the punctuation of the main sentence,
# if the main sentence endswith a punctuation. This is to handle `CARPENTER 3344 011218.txt`.
## "Completion of a minimum 18-month or longer program in carpentry or cabinetmaking and millwork 
### from an accredited institution may be substituted for each year of experience lacking up 
### to a maximum of two years.""
# The idea is that it is different from a subrequirement because it doens't have an itemized initiator, like a. or a)
for aList in line_item:
    for item in list(aList):
        if len(item) > 5:
            if item.split()[0][1] != '.':
                aList.remove(item)
            
line_item

[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or'],
 ['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
  'a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or',
  'b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or',
  'c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or'],
 ['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and',
  'a. Satisfactory completion of four courses, of at

In [52]:
requirement_list = []
for aList in line_item:
    # First item in the list aList may be a requirement
    # requirement_id is the first element in the string
    # requirement_txt is anything after, for example, 1., 2., 3., etc. (2 characters, that's why 2:)
    main_information = aList[0]
    requirement_id   = main_information[0]; requirement_txt = main_information[2:main_information.find(';')]
    # Step 1
    if len(aList) == 1:
        ## Step 1a: Handle case "1. and 2.", "1. or 2."
        if main_information.endswith('and'):
            ### Find next item's requirement id and the text associated with it
            next_list_idx   = line_item.index(aList)+1; next_list     = line_item[next_list_idx]
            next_info       = next_list[0]            ; next_info_txt = next_info[2:next_info.find(';')]
            ### Form a nested requirement and put into requirement list
            nested_reqmnt   = (requirement_id, requirement_txt, next_digit, next_req_txt)
            requirement_list.append(nested_reqmnt)
        ## Step 1b: Handle case "1. or 2."
        else:
            requirement = (requirement_id, requirement_txt, 'A', None)
            requirement_list.append(requirement)
    # Step 2
    elif len(aList) > 1:
        # Step 2a: Handle case "1. a. or b."
        if all([sub_item.endswith('or') for sub_item in aList[1:-1]]):
            all_sub_requirements = aList[1:]
            for sub_requirement in all_sub_requirements:
                requirement_subset_id  = sub_requirement[0].upper()
                requirement_subset_txt = sub_requirement[2:sub_requirement.find(';')]
                nested_reqmnt = (requirement_id, requirement_txt, requirement_subset_id, requirement_subset_txt)
                requirement_list.append(nested_reqmnt)
        # Step 2b: Handle case "3. a. and b."
        else:
            nested_reqmnt = (requirement_id, requirement_txt, 'A', None, aList[1:])
            requirement_list.append(nested_reqmnt)

# Normalize each element in requirement list to length 4
requirement_list = [tup+(None, )*(4-len(tup)) for tup in requirement_list]

# Display result
requirement_list

[('1',
  ' Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems',
  'A',
  None),
 ('2',
  ' Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in',
  'A',
  ' the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases'),
 ('2',
  ' Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in',
  'B',
  ' performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system'),
 ('2',
  ' Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of 

In [53]:
pd.DataFrame(requirement_list)

,0,1,2,3,4
0,1,Graduation from an accredited four-year colle...,A,None,None
1,2,Graduation from an accredited four-year colle...,A,"the development, analysis, implementation or ...",None
2,2,Graduation from an accredited four-year colle...,B,"performing cost benefit, feasibility and requ...",None
3,2,Graduation from an accredited four-year colle...,C,performing system implementation and support ...,None
4,3,Two years of full-time paid experience as a S...,A,None,"[a. Satisfactory completion of four courses, o..."


In [66]:
# Define a helper function
def _base(job, use_df=True):
    '''
    _BASE attempts to parse the structure in REQUIREMENTS/MINIMUM QUALIFICATIONS. 
    It takes in a job and returns a dataframe of information, which supports other functions.
    
    It is intended to use inside other functions (hence started with an underscore), not for stand-alone use.
    '''
    # Each hash # below is the code in one of the cell above. For output/logic, see above cells.
    
    # From REQUIREMENTS/MINIMUM QUALIFICATIONS to PROCESS NOTES is where the information located
    temp = job[job.find('REQUIREMENTS/MINIMUM QUALIFICATIONS'):job.find('PROCESS NOTES')]
    
    # Get the digits that mark one part from the other
    digits_as_str = [sentence[-1] for sentence in temp.split('.') if sentence[-1].isdigit()]
    
    # Separate requirements one from the other.
    line_item = [] # borrow from accounting term. means items in a list
    ## Loop through idx of digits_as_str to get the text from one digit to the other
    for idx in range(len(digits_as_str)-1):
        start = digits_as_str[idx]  ; pos1 = temp.index(start)
        end   = digits_as_str[idx+1]; pos2 = temp.index(end)
        item  = temp[pos1:pos2]     ; line_item.append(item)
    ## Put the last part into the list of all possible requirements
    final_dig    = digits_as_str[-1]; final_pos = temp.index(final_dig)
    final_item = temp[final_pos:] ; line_item.append(final_item)

    # Split further by newline characters
    line_item = [[line for line in item.split('\n') if len(line) > 1] for item in line_item]
    
    # Simplify line_item by throwing away anything after the punctuation 
    # of the main sentence, if the main sentence endswith a punctuation
    for aList in line_item:
        for item in list(aList):
            if len(item) > 5:
                if item.split()[0][1] != '.':
                    aList.remove(item)
    
    # Implementation of the algorithm
    requirement_list = []
    for aList in line_item:
        # First item in the list aList may be a requirement
        # requirement_id is the first element in the string
        # requirement_txt is anything after, for example, 1., 2., 3., etc. (2 characters, that's why 2:)
        main_information = aList[0]
        requirement_id   = main_information[0]; requirement_txt = main_information[2:main_information.find(';')]
        # Step 1
        if len(aList) == 1:
            ## Step 1a: Handle case "1. and 2.", "1. or 2."
            if main_information.endswith('and'):
                ### Find next item's requirement id and the text associated with it
                next_list_idx   = line_item.index(aList)+1; next_list     = line_item[next_list_idx]
                next_info       = next_list[0]            ; next_info_txt = next_info[2:next_info.find(';')]
                ### Form a nested requirement and put into requirement list
                nested_reqmnt   = (requirement_id, requirement_txt, next_digit, next_req_txt)
                requirement_list.append(nested_reqmnt)
            ## Step 1b: Handle case "1. or 2."
            else:
                requirement = (requirement_id, requirement_txt, 'A', None)
                requirement_list.append(requirement)
        # Step 2
        elif len(aList) > 1:
            # Step 2a: Handle case "1. a. or b."
            if all([sub_item.endswith('or') for sub_item in aList[1:-1]]):
                all_sub_requirements = aList[1:]
                for sub_requirement in all_sub_requirements:
                    requirement_subset_id  = sub_requirement[0].upper()
                    requirement_subset_txt = sub_requirement[2:sub_requirement.find(';')]
                    nested_reqmnt = (requirement_id, requirement_txt, requirement_subset_id, requirement_subset_txt)
                    requirement_list.append(nested_reqmnt)
            # Step 2b: Handle case "3. a. and b."
            else:
                nested_reqmnt = (requirement_id, requirement_txt, 'A', None, aList[1:])
                requirement_list.append(nested_reqmnt)

    # Normalize each element in requirement list to length 5
    requirement_list = [tup+(None, )*(5-len(tup)) for tup in requirement_list]

    # Returns
    df = pd.DataFrame(data=requirement_list, columns=['REQUIREMENT_SET_ID',
                                                      'requirement_txt',
                                                      'REQUIREMENT_SUBSET_ID',
                                                      'requirement_subset_txt',
                                                      'mis'])
    return df

# Test
print(_base(job=sa).shape)
_base(job=sa)

(5, 5)


,REQUIREMENT_SET_ID,requirement_txt,REQUIREMENT_SUBSET_ID,requirement_subset_txt,mis
0,1,Graduation from an accredited four-year colle...,A,None,None
1,2,Graduation from an accredited four-year colle...,A,"the development, analysis, implementation or ...",None
2,2,Graduation from an accredited four-year colle...,B,"performing cost benefit, feasibility and requ...",None
3,2,Graduation from an accredited four-year colle...,C,performing system implementation and support ...,None
4,3,Two years of full-time paid experience as a S...,A,None,"[a. Satisfactory completion of four courses, o..."


## Now, let's test it on some other jobs
* ~~CARPENTER 3344 011218.txt~~
* ~~TREE SURGEON 3114 081916 REV 080818.txt~~
* 311 DIRECTOR  9206 041814.txt
* ACCOUNTANT 1513 062218.txt

### CARPENTER 3344 011218.txt. <font color='magenta'>YAHOO!</font>

In [55]:
# Find `CARPENTER 3344 011218.txt`
print(all_txt_files.index('CARPENTER 3344 011218.txt'))

94


In [56]:
# Import file as text
carpenter_path = path + all_txt_files[94]
carpenter = open(carpenter_path, 'rt').read()
carpenter

'CARPENTER\n\nClass Code: 3344\nOpen Date: 01-12-18\n\t                                 (Exam Open to All, including Current City Employees)\n\nANNUAL SALARY\n\n$86,312 (flat-rated)\nThe salary in the Department of Water and Power is $93,542 (flat-rated)\n\nNOTES:\n\n1. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to: http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.\n2. Annual salary is at the start of the pay range. The current salary range is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer.\n\nDUTIES\n\nA Carpenter does skilled rough and finish carpentry in building and repairing wood structures and articles; work may include forming for the placement of concrete.\n\nREQUIREMENTS/MINIMUM QUALIFICATIONS\n\n1. Completion of an apprenticeship as a carpenter or cabinetmaker; or\n2. Six years of full-time paid experience performing carpenter or cabinetmaker work.\n\nCom

In [58]:
_base(job=carpenter)
# Oops!

AssertionError: 5 columns passed, passed data had 4 columns

In [59]:
# Locate the information
temp = carpenter[carpenter.find('REQUIREMENTS/MINIMUM QUALIFICATIONS'):carpenter.find('PROCESS NOTES')]
temp

'REQUIREMENTS/MINIMUM QUALIFICATIONS\n\n1. Completion of an apprenticeship as a carpenter or cabinetmaker; or\n2. Six years of full-time paid experience performing carpenter or cabinetmaker work.\n\nCompletion of a minimum 18-month or longer program in carpentry or cabinetmaking and millwork from an accredited institution may be substituted for each year of experience lacking up to a maximum of two years.\n\n'

In [60]:
# Get the digits
digits_as_str = [sentence[-1] for sentence in temp.split('.') if sentence[-1].isdigit()]
digits_as_str

['1', '2']

In [61]:
# Separate requirements one from the other.
## Initialization
line_item = [] # borrow from accounting term. means items in a list
## Loop through idx of digits_as_str to get the text from one digit to the other
for idx in range(len(digits_as_str)-1):
    start = digits_as_str[idx]  ; pos1 = temp.index(start)
    end   = digits_as_str[idx+1]; pos2 = temp.index(end)
    item  = temp[pos1:pos2]     ; line_item.append(item)
## Put the last part into the list of all possible requirements
final_dig    = digits_as_str[-1]; final_pos = temp.index(final_dig)
final_item = temp[final_pos:] ; line_item.append(final_item)
## Display results
line_item

['1. Completion of an apprenticeship as a carpenter or cabinetmaker; or\n',
 '2. Six years of full-time paid experience performing carpenter or cabinetmaker work.\n\nCompletion of a minimum 18-month or longer program in carpentry or cabinetmaking and millwork from an accredited institution may be substituted for each year of experience lacking up to a maximum of two years.\n\n']

In [62]:
# Split further by newline characters
line_item = [[line for line in item.split('\n') if len(line) > 1] for item in line_item]
line_item

[['1. Completion of an apprenticeship as a carpenter or cabinetmaker; or'],
 ['2. Six years of full-time paid experience performing carpenter or cabinetmaker work.',
  'Completion of a minimum 18-month or longer program in carpentry or cabinetmaking and millwork from an accredited institution may be substituted for each year of experience lacking up to a maximum of two years.']]

In [63]:
# Simplify line_item by throwing away anything after the punctuation of the main sentence,
# if the main sentence endswith a punctuation. This is to handle `CARPENTER 3344 011218.txt`.
## "Completion of a minimum 18-month or longer program in carpentry or cabinetmaking and millwork 
### from an accredited institution may be substituted for each year of experience lacking up 
### to a maximum of two years.""
# The idea is that it is different from a subrequirement because it doens't have an itemized initiator, like a. or a)
for aList in line_item:
    for item in list(aList):
        if len(item) > 5:
            if item.split()[0][1] != '.':
                aList.remove(item)
            
line_item

[['1. Completion of an apprenticeship as a carpenter or cabinetmaker; or'],
 ['2. Six years of full-time paid experience performing carpenter or cabinetmaker work.']]

In [67]:
requirement_list = []
for aList in line_item:
    # First item in the list aList may be a requirement
    # requirement_id is the first element in the string
    # requirement_txt is anything after, for example, 1., 2., 3., etc. (2 characters, that's why 2:)
    main_information = aList[0]
    requirement_id   = main_information[0]; requirement_txt = main_information[2:main_information.find(';')]
    # Step 1
    if len(aList) == 1:
        ## Step 1a: Handle case "1. and 2.", "1. or 2."
        if main_information.endswith('and'):
            ### Find next item's requirement id and the text associated with it
            next_list_idx   = line_item.index(aList)+1; next_list     = line_item[next_list_idx]
            next_info       = next_list[0]            ; next_info_txt = next_info[2:next_info.find(';')]
            ### Form a nested requirement and put into requirement list
            nested_reqmnt   = (requirement_id, requirement_txt, next_digit, next_req_txt)
            requirement_list.append(nested_reqmnt)
        ## Step 1b: Handle case "1. or 2."
        else:
            requirement = (requirement_id, requirement_txt, 'A', None)
            requirement_list.append(requirement)
    # Step 2
    elif len(aList) > 1:
        # Step 2a: Handle case "1. a. or b."
        if all([sub_item.endswith('or') for sub_item in aList[1:-1]]):
            all_sub_requirements = aList[1:]
            for sub_requirement in all_sub_requirements:
                requirement_subset_id  = sub_requirement[0].upper()
                requirement_subset_txt = sub_requirement[2:sub_requirement.find(';')]
                nested_reqmnt = (requirement_id, requirement_txt, requirement_subset_id, requirement_subset_txt)
                requirement_list.append(nested_reqmnt)
        # Step 2b: Handle case "3. a. and b."
        else:
            nested_reqmnt = (requirement_id, requirement_txt, 'A', None, aList[1:])
            requirement_list.append(nested_reqmnt)

# Normalize each element in requirement list to length 5
requirement_list = [tup+(None, )*(5-len(tup)) for tup in requirement_list]

# Display result
requirement_list

[('1',
  ' Completion of an apprenticeship as a carpenter or cabinetmaker',
  'A',
  None,
  None),
 ('2',
  ' Six years of full-time paid experience performing carpenter or cabinetmaker work',
  'A',
  None,
  None)]

In [68]:
pd.DataFrame(requirement_list)

,0,1,2,3,4
0,1,Completion of an apprenticeship as a carpente...,A,None,None
1,2,Six years of full-time paid experience perfor...,A,None,None


**Fix the `_base` function: Normalize to length 5**

In [69]:
_base(job=carpenter)

,REQUIREMENT_SET_ID,requirement_txt,REQUIREMENT_SUBSET_ID,requirement_subset_txt,mis
0,1,Completion of an apprenticeship as a carpente...,A,None,None
1,2,Six years of full-time paid experience perfor...,A,None,None


### TREE SURGEON 3114 081916 REV 080818.txt <font color='magenta'>YAHOO!</font>

In [70]:
# Find `TREE SURGEON 3114 081916 REV 080818.txt`
print(all_txt_files.index('TREE SURGEON 3114 081916 REV 080818.txt'))

628


In [72]:
# Import file as text
tree_surgeon_path = path + all_txt_files[628]
tree_sugeon = open(tree_surgeon_path, 'rt').read()
tree_sugeon

'TREE SURGEON\n\nClass Code:      3114\nOpen Date:  08-19-16\nREVISED: 08-08-18\n(Exam Open to All, including Current City Employees)\n\nANNUAL SALARY\n          \n$47,898 to $70,052 \nThe salary range for positions in the Department of Water and Power is $72,328 to $89,864\n\nNOTES:\n\n1. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.\n2. Annual salary is at the start of the pay range. The current salary range is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer.\n\nDUTIES\n\nA Tree Surgeon climbs, prunes, trims, treats, sprays, plants, fertilizes, transplants and removes trees along City streets and highways, in City parks, over or on private property, and on public grounds to eliminate hazardous conditions; prevent interference with traffic, traffic signals and high and low voltage conductors and equipment; and to pr

In [73]:
_base(job=tree_sugeon)

,REQUIREMENT_SET_ID,requirement_txt,REQUIREMENT_SUBSET_ID,requirement_subset_txt,mis
0,1,One year of full-time paid experience with th...,A,None,None
1,2,One year of full-time paid experience as a tr...,A,None,None


### 311 DIRECTOR  9206 041814.txt

In [74]:
# Find `311 DIRECTOR  9206 041814.txt`
print(all_txt_files.index('311 DIRECTOR  9206 041814.txt'))

0


In [75]:
# Import file as text
director311_path = path + all_txt_files[0]
director311 = open(director311_path, 'rt').read()
director311

'311 DIRECTOR\nClass Code:       9206\nOpen Date:  04-18-14\nANNUAL SALARY \n\n$125,175 to $155,514 \n\nDUTIES\n\nA 311 Director is responsible for the successful operation and expansion of the 311 Call Center in the Information Technology Agency (ITA) which answers call from constituents regarding Citywide services provided by City departments; works to ensure the efficient and effective resolution of any issues that may arise; plans, directs, hires, coaches, and coordinates a large staff of professional, technical and clerical employees engaged in the implementation, administration, and operations of the City\'s 311 Call Center; applies sound supervisor principles and techniques in building and maintaining and effective work force; fulfills equal opportunity responsibilities; and does related work.\n\nREQUIREMENTS\n\n1. One year of full-time paid experience as a Senior Management Analyst with the City of Los Angeles or in a class which is at least at the level which provides professi

In [77]:
_base(job=director311)
# Oops!

IndexError: string index out of range

In [85]:
# Locate the information
temp = director311[director311.find('REQUIREMENTS/MINIMUM QUALIFICATIONS'):director311.find('PROCESS NOTES')]
temp

''

# <font size=8, color='red'> Culprit found!</font> Need to use regex for information location

### ACCOUNTANT 1513 062218.txt

In [80]:
# Find `ACCOUNTANT 1513 062218.txt`
print(all_txt_files.index('ACCOUNTANT 1513 062218.txt'))

1


In [82]:
# Import file as text
accountant_path = path + all_txt_files[1]
accountant = open(accountant_path, 'rt').read()
accountant

'ACCOUNTANT\n\nClass Code:       1513\nOpen Date:  06-22-18\n(Exam Open to All, including Current City Employees)\n\nANNUAL SALARY\n\n$49,903 to $72,996 and $55,019 to $80,472\n\nNOTES:\n\n1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.\n2. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.\n3. Annual salary is at the start of the pay range. The current salary range is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer.\n\nDUTIES\n\nAn Accountant does professional accounting work in the analysis, preparation, maintenance, control, and reconciliation of financial records and reports in accordance with the principles of governmental and public accounting.\n\nREQUIREMENT/MINIMUM QUALIFICATION\n\nGraduation from an accredited four-year college or university and at least 

In [84]:
_base(job=accountant)
# Oops

IndexError: string index out of range

In [126]:
# Locate the information
temp = accountant[accountant.find('REQUIREMENTS/MINIMUM QUALIFICATIONS'):accountant.find('PROCESS NOTES')]
temp

''

In [123]:
# Get the digits
digits_as_str = [sentence[-1] for sentence in temp.split('.') if sentence[-1].isdigit()]
digits_as_str

[]

In [124]:
# Note that temp doesn't have itemized initiator in it. So we'll need to manually add a number 1 in it.
if len(digits_as_str) == 0:
    digits_as_str.append('1')
    digits_as_str.append('2')

digits_as_str

['1', '2']

In [125]:
# Separate requirements one from the other.
## Initialization
line_item = [] # borrow from accounting term. means items in a list
## Loop through idx of digits_as_str to get the text from one digit to the other
for idx in range(len(digits_as_str)-1):
    start = digits_as_str[idx]  ; pos1 = temp.index(start)
    end   = digits_as_str[idx+1]; pos2 = temp.index(end)
    item  = temp[pos1:pos2]     ; line_item.append(item)
## Put the last part into the list of all possible requirements
final_dig    = digits_as_str[-1]; final_pos = temp.index(final_dig)
final_item = temp[final_pos:] ; line_item.append(final_item)
# Display results
line_item

0


ValueError: substring not found

# <font size='8' color='red'>I'M STUCK HERE. TO BE CONTINUED </font> 